# Lista 13 (8 pkt.)

We wszystkich poniższych zadaniach będziemy używać **OpenSSL** https://www.openssl.org/.

## Krótki tutorial:

### generowanie klucza prywatnego
```
openssl genpkey -algorithm RSA -pkeopt rsa_keygen_bits:2048 -pkeyopt rsa_keygen_pubexp:3 -out privkey.pem
```
```-algorithm``` wybiera algorytm do generacji klucza, tu ```RSA```

```-pkeyopt rsa_keygen_bits:2048``` ustala długość klucza, tu ```2048``` bitów

```-pkeyopt rsa_keygen_pubexp:3``` ustala wartość $e$, tu ```3```

```-out privkey.pem``` zapisuje klucz prywatny w pliku ```privkey.pem```
### generowanie klucza publicznego
```
openssl pkey -in privkey.pem -pubout -out pubkey.pem
```
```-in privkey.pem``` wczytuje plik z kluczem prywatnym

```-pubout``` ustala, że chcemy uzyskać klucz publiczny

```-out pubkey.pem``` zapisuje klucz publiczny w pliku ```pubkey.pem```

### Diffy-Helman
- generowanie publicznych parametrów $p$ i $g$
```
openssl genpkey -genparam -algorithm DH -out dh_params.pem
```
- generowanie klucza prywatnego $a$
```
openssl genpkey -paramfile dh_params.pem -out privkey.pem
```
- generowanie klucza publicznego $g^a$
```
openssl pkey -in privkey.pem -pubout -out pubkey.pem
```
- generowanie wspólnego sekretu $k=(g^b)^a=(g^a)^b$
```
openssl pkeyutl -derive -inkey privkey_A.pem -peerkey pubkey_B.pem -out secret.bin
```

### wyświetlanie w przyjaznej formie
- klucz prywatny
```
openssl pkey -in privkey.pem -text -noout
```
- klucz publiczny
```
openssl pkey -pubin -in pubkey.pem -text -noout
```
- certyfikat
```
openssl x509 -in user.crt -text -noout
```
- parametry Diffiego-Helmana
```
openssl pkeyparam -in dh_params.pem -text -noout
```
### generowanie żądania popisania certyfikatu
```
openssl req -new -key privkey.pem -out req.csr
```
```-key privkey.pem``` wskazuje plik z naszym kluczem prywatnym

```-out req.csr``` zapisuje żądanie certyfikatu do pliku ```req.csr``

po wpisaniu tej komendy pojawi się seria pytań o dane do certyfikatu, można wpisać jakieś rzeczy typu PL, Cracow, UJ itd., można też wpisać '.' (kropkę) i w ten sposób pominąć dane pole

Uwaga: aby certyfikat przechodził później weryfikację bez problemu, dane wpisane w certyfikat użytkownika, i dane wpisane w certyfikat **CA** nie mogą być identyczne.

### generowanie samopodpisanego certyfikatu przez **CA**
```
openssl req -x509 -new -nodes -key rootkey.pem -sha256 -days 1024 -out root.crt
```

```-key rootkey.pem``` wskazuje klucz prywatny **CA**

```-sha256``` wybiera funkcję hashującą używaną do podpisu, tu **SHA 256**

```-days 1024``` ustala termin ważności certyfikatu

```-out root.crt``` zapisuje certyfikat do pliku ```root.crt```

### wygenerowanie i podpisanie certyfiaktu przez **CA** na podstawie żądania
```
openssl x509 -req -in req.csr -CA root.crt -CAkey rootkey.pem -CAcreateserial -out user.crt -days 500 -sha256
```
```-req -in req.csr``` wskazuje żądanie

```-CA root.crt``` wskazuje klucz prywatny **CA**

```-CAcreateserial``` tworzy numer seryjny cerrtyfikatu

```-out user.crt``` zapisuje podpisany certyfikat do pliku ```user.crt```

```-days 500``` ustala termin ważności certyfikatu, tu ```500``` dni

```-sha256``` wybiera funkcję hashującą używaną do podpisu, tu **SHA 256**
### weryfikacja certyfikatu
```
openssl verify -CAfile root.crt user.crt
```
```-CAfile root.crt``` wskazuje certyfikat **CA** na podstawie którego weryfikujemy certyfiakt użytkownika

```user.crt``` certyfikat, który weryfikujemy
### wydobycie klucza publicznego użytkownika z jego certyfikatu
```
openssl x509 -pubkey -in user.crt -noout > pubkey.pem
```
```-pubkey``` wskazuje, że wyciągamy klucz publiczny

```-in user.crt``` wskazuje plik z certyfikatem

```> pubkey.pem``` zapisuje uzyskany klucz do pliku ```pubkey.pem```

### szyfrowanie pliku za pomocą szyfru asymetrycznego
```
openssl pkeyutl -encrypt -in file.txt -pubin -inkey pubkey.pem -out file.enc
```
```-encrypt``` włącza tryb szyfrowania

```-in file.txt``` wskazuje plik do zaszyfrowania

```-pubin -inkey pubkey.pem``` wskazuje klucz publiczny, którym szyfrujemy

```-out file.enc``` zapsiuje zaszyfrowany plik do pliku ```file.enc```
### deszyfrowanie pliku za pomocą szyfru asymetrycznego
```
openssl pkeyutl -decrypt -in file.enc -inkey privkey.pem -out file.txt
```
```-decrypt``` włącza tryb deszyfrowania

```-in file.enc``` wskazuje plik do deszyfrowania

```-inkey pubkey.pem``` wskazuje klucz prywatny, którym deszyfrujemy

```-out fil.txt``` zapsiuje odszyfrowany plik do pliku ```file.txt```

### szyfrowanie pliku za pomocą szyfru symetrycznego
- na podstawie hasła
```
openssl enc -aes-256-cbc -p -pbkdf2 -in file.txt -out file_enc.bin
```
pojawi się zapytanie o hasło, można też dodać ```-pass file:key_pass.pem``` aby pobrać hasło z pliku ```key_pass.pem```

```-aes-256-cbc``` wybiera jako szyfr **AES** z kluczem **256** bitowym w trybie pracy **CBC**

```-p``` wypisuje na ekranie klucz, sól i wektor inicjujący

```--pbkdf2``` ustala metodę używaną przy generowaniu klucza

```-in file.txt``` plik do zaszyfrowania

```file_enc.bin``` zaszyfrowany plik
- na podstawie klucza
```
openssl enc -aes-256-cbc -K XXXXXXXXX -iv XXXXXXXX -p -in file.txt -out file_enc.bin
```
```-aes-256-cbc``` wybiera jako szyfr **AES** z kluczem **256** bitowym w trybie pracy **CBC**

```-K XXXXXXXXX``` ustala klucz na ```XXXXXXXXX```, format szesnastkowy, odpowiednia długość do szyfru

```-iv XXXXXXXXX``` ustala wektor inicjujący na ```XXXXXXXXX```, format szesnastkowy, odpowiednia długość do szyfru

```-p``` wypisuje na ekranie klucz, sól i wektor inicjujący

```-in file.txt``` plik do zaszyfrowania

```file_enc.bin``` zaszyfrowany plik

### deszyfrowanie pliku za pomocą szyfru symetrycznego
- na podstawie hasła
```
openssl enc -aes-256-cbc -d -p -pbkdf2 -in file_enc.bin -out file.txt
```
pojawi się zapytanie o hasło, można też dodać ```-pass file:key_pass.pem``` aby pobrać hasło z pliku ```key_pass.pem```

```-aes-256-cbc``` wybiera jako szyfr **AES** z kluczem **256** bitowym w trybie pracy **CBC**

```-d``` wybiera tryb deszyfracji

```-pass file:key_pass.pem``` wybiera deszyfrowanie na podstawie klucza wygenerowanego z hasła znajdującego się w pliku ```key_pass.pem```

```-p``` wypisuje na ekranie klucz, sól i wektor inicjujący

```-pbkdf2``` ustala funkcję hashującą używaną przy generowaniu klucza

```-in file_enc.bin``` plik do deszyfrowania

```file.txt``` odszyfrowany plik
- na podstawie klucza
```
openssl enc -aes-256-cbc -d -K XXXXXXXXX -iv XXXXXXXX -p -in file.enc -out file.txt
```
```-aes-256-cbc``` wybiera jako szyfr **AES** z kluczem **256** bitowym w trybie pracy **CBC**

```-d``` wybiera tryb deszyfracji

```-K XXXXXXXXX``` ustala klucz na ```XXXXXXXXX```, format szesnastkowy, odpowiednia długość do szyfru

```-iv XXXXXXXXX``` ustala wektor inicjujący na ```XXXXXXXXX```, format szesnastkowy, odpowiednia długość do szyfru

```-p``` wypisuje na ekranie klucz, sól i wektor inicjujący

```-in file.enc``` plik do odszyfrowania

```file.txt``` odszyfrowany plik

### generowanie podpisu elektronicznego
```
openssl dgst -sha1 -sign privkey.pem -out signature.bin file.txt
```
```-sha1``` wybiera funkcję hashującą **SHA 1**

```-sign privkey.pem``` podpisuje kluczem prywatnym ```privkey.pem```

```-out signature.bin``` zapisuje podpis do pliku binarnego ```signature.bin```

```file.txt``` podpisywany plik
### weryfikacja podpisu elektronicznego
```
openssl dgst -sha1 -verify pubkey.pem -signature signature.bin file.txt
```
```-sha1``` wybiera funkcję hashującą **SHA 1**

```-verify pubkey.pem``` weryfikuje podpis kluczem publicznm ```pubkey.pem```

```-signature signature.bin``` wskazuje plik z podpisem do weryfikacji

```file.txt``` plik, którego podpis weryfikujemy

### generowanie pseudo-losowych bajtów
```
openssl rand -base64 32 -out symkey.pem
```
- ```-base64``` ustala kodowanie na base64

- ```32``` liczba bajtów do wygenerowania

- ```-out symkey.pem``` zapisuje ciąg bajtów do symkey.pem


## Zadanie 1 (1 pkt.)

Zdeszyfruj plik **AESencryptedCBC.enc** za pomocą klucza:

8541F781259ADA8631F18A9A8E97771BFE395CF7ACBE5F77

oraz wektora inicjującego:

99B54E58FF44F7599465E6888A3C6C1E

w trybie **CBC** z kluczem **192** bitowym.

In [23]:
import os
os.environ["LC_ALL"] = "C"
os.environ["LANG"] = "C"


In [24]:
!openssl version
!openssl version -a
!pwd
!ls -la

!export LC_ALL=C; export LANG=C; ls -la AESencrypted.dec


OpenSSL 3.0.13 30 Jan 2024 (Library: OpenSSL 3.0.13 30 Jan 2024)
OpenSSL 3.0.13 30 Jan 2024 (Library: OpenSSL 3.0.13 30 Jan 2024)
built on: Thu Sep 18 11:12:48 2025 UTC
platform: debian-amd64
options:  bn(64,64)
compiler: gcc -fPIC -pthread -m64 -Wa,--noexecstack -Wall -fzero-call-used-regs=used-gpr -DOPENSSL_TLS_SECURITY_LEVEL=2 -Wa,--noexecstack -g -O2 -fno-omit-frame-pointer -mno-omit-leaf-frame-pointer -ffile-prefix-map=/build/openssl-S7huCI/openssl-3.0.13=. -fstack-protector-strong -fstack-clash-protection -Wformat -Werror=format-security -fcf-protection -fdebug-prefix-map=/build/openssl-S7huCI/openssl-3.0.13=/usr/src/openssl-3.0.13-0ubuntu3.6 -DOPENSSL_USE_NODELETE -DL_ENDIAN -DOPENSSL_PIC -DOPENSSL_BUILDING_OPENSSL -DNDEBUG -Wdate-time -D_FORTIFY_SOURCE=3
OPENSSLDIR: "/usr/lib/ssl"
ENGINESDIR: "/usr/lib/x86_64-linux-gnu/engines-3"
MODULESDIR: "/usr/lib/x86_64-linux-gnu/ossl-modules"
Seeding source: os-specific
CPUINFO: OPENSSL_ia32cap=0xfffa32235f8bffff:0x184007a4219c27ab
/mnt/d

In [25]:
!ls -la AESencrypted.enc

!openssl enc -aes-192-cbc -d -K 8541F781259ADA8631F18A9A8E97771BFE395CF7ACBE5F77 -iv 99B54E58FF44F7599465E6888A3C6C1E -in AESencrypted.enc -out AESencrypted.dec

!ls -la AESencrypted.dec


-rwxrwxrwx 1 ziolo ziolo 3392 Jan 13 21:32 AESencrypted.enc
-rwxrwxrwx 1 ziolo ziolo 3388 Jan 13 21:47 AESencrypted.dec


In [2]:
!file AESencrypted.dec

!head -n 30 AESencrypted.dec


from pathlib import Path
p = Path("AESencrypted.dec")
data = p.read_bytes()
print("bytes:", len(data))

print(data[:500].decode("utf-8"))


/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
AESencrypted.dec: Unicode text, UTF-8 text, with CRLF line terminators
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Juliusz Słowacki

Hymn o zachodzie słońca na morzu



Smutno mi, Boże! Dla mnie na zachodzie
Rozlałeś tęczę blasków promienistą;
Przede mną gasisz w lazurowej wodzie
    Gwiazdę ognistą…
Choć mi tak niebo Ty złocisz i morze,
    Smutno mi, Boże!
Jak puste kłosy z podniesioną głową,
Stoję rozkoszy próżen i dosytu…
Dla obcych ludzi mam twarz jednakową,
    Ciszę błękitu.
Ale przed Tobą głąb serca otworzę:
    Smutno mi, Boże!
Jako na matki odejście się żali
Mała dziecina, tak ja płaczu bliski,
Patrząc na słońce, co mi rzuca z fali
    Ostatnie błyski,
Choć wiem, że jutro błyśnie nowe zorze,
    Smutno mi, Boże!
Dzisiaj na wielkiem morzu obłąkany,
Sto mil od brzegu i sto mil przed brzegiem,
Widziałem lotne w powietrzu bociany
    Długim szeregiem.
Żem je znał kiedyś na pols

## Zadanie 2 (1 pkt.)

Zaszyfruj plik **kamien.txt** za pomocą hasła w trybie **OFB** z kluczem **256** bitowym.

In [27]:
from pathlib import Path
Path("haslo.txt").write_text("MojeHaslo123!_dlugie\n", encoding="utf-8")


21

In [28]:
!rm -f kamien.ofb.enc
!openssl enc -aes-256-ofb -pbkdf2 -salt -in kamien.txt -out kamien.ofb.enc -pass file:haslo.txt
!ls -la kamien.ofb.enc


-rwxrwxrwx 1 ziolo ziolo 944 Jan 13  2026 kamien.ofb.enc


In [29]:
!rm -f kamien.dec.txt
!openssl enc -aes-256-ofb -d -pbkdf2 -in kamien.ofb.enc -out kamien.dec.txt -pass file:haslo.txt

!diff -u kamien.txt kamien.dec.txt || true


## Zadanie 3 (1 pkt.)

Stwórz dwa foldery, **Alice** oraz **Bob**, i przeprowadź protokół uzgadniania klucza za pomocą metody Diffiego-Helmana pomiędzy tymi folderami. Gdy uzsykasz w obu folderach plik **secret.bin**, który jest plikiem binarnym zawierającym uzgodniony ciąg bitów pomiędzy Alicją i Bobem, zaszyfruj plik **rozwijajac_rilkego.txt** w folderze Alicji podając jako hasło plik **secret.bin** a następnie zdeszyfruj go w folderze Boba za pomocą jego pliku **secret.bin**. Użyj AES 256 w dowolnym trybie.

In [30]:
!rm -rf Alice Bob dh_params.pem
!mkdir Alice Bob
!ls -la


total 80
drwxrwxrwx 1 ziolo ziolo  4096 Jan 13 21:49 .
drwxrwxrwx 1 ziolo ziolo  4096 Jan 13 20:42 ..
-rwxrwxrwx 1 ziolo ziolo  3388 Jan 13 21:47 AESencrypted.dec
-rwxrwxrwx 1 ziolo ziolo  3392 Jan 13 21:32 AESencrypted.enc
drwxrwxrwx 1 ziolo ziolo  4096 Jan 13 21:49 Alice
drwxrwxrwx 1 ziolo ziolo  4096 Jan 13 21:49 Bob
-rwxrwxrwx 1 ziolo ziolo 25113 Jan 13 21:49 Lista11.ipynb
-rwxrwxrwx 1 ziolo ziolo  1046 Jan 13 20:42 ToJa1.crt
-rwxrwxrwx 1 ziolo ziolo  1046 Jan 13 20:42 ToJa2.crt
-rwxrwxrwx 1 ziolo ziolo  1078 Jan 13 20:42 ToJa3.crt
-rwxrwxrwx 1 ziolo ziolo  2352 Jan 13 20:41 bajka.txt
-rwxrwxrwx 1 ziolo ziolo    21 Jan 13 21:47 haslo.txt
-rwxrwxrwx 1 ziolo ziolo   928 Jan 13 21:47 kamien.dec.txt
-rwxrwxrwx 1 ziolo ziolo   944 Jan 13 21:47 kamien.ofb.enc
-rwxrwxrwx 1 ziolo ziolo   928 Jan 13 20:41 kamien.txt
-rwxrwxrwx 1 ziolo ziolo  1436 Jan 13 20:42 plik1.txt
-rwxrwxrwx 1 ziolo ziolo  1432 Jan 13 20:42 plik2.txt
-rwxrwxrwx 1 ziolo ziolo  1433 Jan 13 20:42 plik3.txt
-rwxrwxrwx 1 zi

In [31]:
# Parametry DH (publiczne)
!openssl genpkey -genparam -algorithm DH -out dh_params.pem
!cp dh_params.pem Alice/
!cp dh_params.pem Bob/

# Alice: klucz prywatny i publiczny
%cd Alice
!openssl genpkey -paramfile dh_params.pem -out privkey_A.pem
!openssl pkey -in privkey_A.pem -pubout -out pubkey_A.pem
%cd ..

# Bob: klucz prywatny i publiczny
%cd Bob
!openssl genpkey -paramfile dh_params.pem -out privkey_B.pem
!openssl pkey -in privkey_B.pem -pubout -out pubkey_B.pem
%cd ..

# "Wymiana" kluczy publicznych między folderami
!cp Alice/pubkey_A.pem Bob/
!cp Bob/pubkey_B.pem Alice/

# Wyprowadzenie wspólnego sekretu (w obu miejscach powinien być taki sam)
%cd Alice
!openssl pkeyutl -derive -inkey privkey_A.pem -peerkey pubkey_B.pem -out secret.bin
%cd ..

%cd Bob
!openssl pkeyutl -derive -inkey privkey_B.pem -peerkey pubkey_A.pem -out secret.bin
%cd ..


.........+......................................................................................................................................................................................................................................................+............................................................................................................................................................................................................+..........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [32]:
!sha256sum Alice/secret.bin Bob/secret.bin


26b1472de368894629819eed83bb95e56555037459557e4473c36bea1134b80b  Alice/secret.bin
26b1472de368894629819eed83bb95e56555037459557e4473c36bea1134b80b  Bob/secret.bin


In [33]:
!cp rozwijajac_rilkego.txt Alice/

%cd Alice
!openssl enc -aes-256-cbc -pbkdf2 -salt -in rozwijajac_rilkego.txt -out rilke.enc -pass file:secret.bin
%cd ..

!cp Alice/rilke.enc Bob/


/mnt/d/Studia/Kryptografia/zestaw11/Alice
/mnt/d/Studia/Kryptografia/zestaw11


In [34]:
%cd Bob
!openssl enc -aes-256-cbc -d -pbkdf2 -in rilke.enc -out rilke.dec.txt -pass file:secret.bin

!diff -u ../rozwijajac_rilkego.txt rilke.dec.txt || true
%cd ..


/mnt/d/Studia/Kryptografia/zestaw11/Bob
/mnt/d/Studia/Kryptografia/zestaw11


## Zadanie 4 (1 pkt.)

Klikając w kłodkę w przeglądarce internetowej otwórz certyfikaty następujących stron:

- Wikipedia
- Google
- Strona tego kursu kryptografii

Odczytaj z tych certyfikatów następujące informacje:
- data ważności certyfikatu
- rodzaj używanego algorytmu asymetrycznego
- klucz publiczny
- rodzaj używanego algorytmu do podpisu

### Zadanie 4

**Wikipedia**
- Ważny od: poniedziałek,           13.03.2024, 01:00:00 CET
- Ważny do: niedziela,              13.03.2027, 00:59:59 CET
- Algorytm asymetryczny:            Klucz publiczny oparty na krzywej eliptycznej
- Klucz publiczny:                  00 04 D3 DC 38 AE EA 0E 52 38 9B C3 74 7F DE 36
                                    0B 0F 94 42 03 51 B4 7E 74 77 51 D0 94 1B CD 17
                                    8D D0 D8 44 7A D4 E0 58 AB BE DB F7 B0 11 A5 90
                                    08 2A 14 73 7F 7A 87 06 38 0A E3 B6 A1 AA 60 3C
                                    E9 8E
- Algorytm podpisu:                 PKCS #1 SHA-256 z szyfrowaniem RSA

**Google**
- Ważny od:                         3.12.2025, 16:59:32 CET
- Ważny do:                         25.02.2026, 16:59:31 CET
- Algorytm asymetryczny:            Klucz publiczny oparty na krzywej eliptycznej
- Klucz publiczny:                  00 04 B8 18 E1 AD 25 AB 6E E3 24 BE BC 15 43 A8
                                    79 5D CE 0D 11 BC 25 B7 51 99 76 EC 97 F6 92 2B
                                    30 2B 90 F3 89 24 EF 36 DE 3C 70 A0 15 D3 FB F1
                                    F9 F8 95 D9 39 77 F8 19 F4 6B 4F B4 37 9F E5 09
                                    3D 7E
- Algorytm podpisu:                 Podpis X9.62 ECDSA za pomocą SHA-256

**Strona kursu ** - nie działa
- Ważny od: ...
- Ważny do: ...
- Algorytm asymetryczny: ...
- Klucz publiczny: ...
- Algorytm podpisu: ...


## Zadanie 5 (1 pkt.)

Spróbuj zaszyfrować plik bajka.txt za pomocą RSA. Co się dzieje?

In [36]:
!rm -f privkey_rsa.pem pubkey_rsa.pem bajka.rsa.enc

!openssl genpkey -algorithm RSA -pkeyopt rsa_keygen_bits:2048 -out privkey_rsa.pem
!openssl pkey -in privkey_rsa.pem -pubout -out pubkey_rsa.pem

!ls -la privkey_rsa.pem pubkey_rsa.pem


.....+.......+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*....+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*..................+.+.....+.....................+...+.+...+.....+.......+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
...+....+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*.+..+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*.................+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
-rwxrwxrwx 1 ziolo ziolo 1704 Jan 13 22:21 privkey_rsa.pem
-rwxrwxrwx 1 ziolo ziolo  451 Jan 13 22:21 pubkey_rsa.pem


In [37]:
!openssl pkeyutl -encrypt -pubin -inkey pubkey_rsa.pem -in bajka.txt -out bajka.rsa.enc


Public Key operation error
40172F5C767F0000:error:0200006E:rsa routines:ossl_rsa_padding_add_PKCS1_type_2_ex:data too large for key size:../crypto/rsa/rsa_pk1.c:133:


RSA nie nadaje się do szyfrowania długich plików, bo ma ograniczenie długości wiadomości

## Zadanie 6 (1 pkt.)

Sprawdź, który z certyfikatów **ToJa1.crt**, **ToJa2.crt** czy **ToJa3.crt** jest prawdziwy. Certyfikat zaufanej trzeciej strony to **root.crt**.

Następnie wyekstrahuj klucz publiczny z prawdziwego certyfikatu i sprawdź, który z plików **plik1.txt**, **plik2.txt** czy **plik3.txt**, zostały podpisane przez właściciela certyfikatu. Podpisy plików znajdują się odpowiednio w plikach **signature1.bin**, **signature2.bin** i **signature3.bin**. Pliki podpisano za pomocą **SHA 1**.

In [38]:
!openssl verify -CAfile root.crt ToJa1.crt
!openssl verify -CAfile root.crt ToJa2.crt
!openssl verify -CAfile root.crt ToJa3.crt


C = Pl, ST = KRK, O = UJ
error 7 at 0 depth lookup: certificate signature failure
error ToJa1.crt: verification failed
40D74C08577F0000:error:02000068:rsa routines:ossl_rsa_verify:bad signature:../crypto/rsa/rsa_sign.c:430:
40D74C08577F0000:error:1C880004:Provider routines:rsa_verify:RSA lib:../providers/implementations/signature/rsa_sig.c:774:
40D74C08577F0000:error:06880006:asn1 encoding routines:ASN1_item_verify_ctx:EVP lib:../crypto/asn1/a_verify.c:217:
ToJa2.crt: OK
C = PL, ST = KRK, O = Internet Widgits Pty Ltd
error 7 at 0 depth lookup: certificate signature failure
error ToJa3.crt: verification failed
409732ECE47F0000:error:02000068:rsa routines:ossl_rsa_verify:bad signature:../crypto/rsa/rsa_sign.c:430:
409732ECE47F0000:error:1C880004:Provider routines:rsa_verify:RSA lib:../providers/implementations/signature/rsa_sig.c:774:
409732ECE47F0000:error:06880006:asn1 encoding routines:ASN1_item_verify_ctx:EVP lib:../crypto/asn1/a_verify.c:217:


In [39]:
!openssl x509 -pubkey -in ToJa2.crt -noout > pubkey_ToJa.pem
!openssl pkey -pubin -in pubkey_ToJa.pem -text -noout


Public-Key: (2048 bit)
Modulus:
    00:a8:64:ba:a4:81:6d:77:be:28:b8:f5:60:f0:dd:
    f3:e4:55:2a:1e:2a:67:71:8d:76:37:91:81:c7:65:
    e3:ef:af:45:1f:7a:57:93:c2:53:03:04:1e:80:fc:
    58:8a:c2:91:bb:10:bc:93:35:c8:08:1d:81:62:24:
    82:d9:f1:82:27:ad:be:92:9e:36:c5:fe:76:5a:d5:
    44:ab:70:2b:c4:31:47:72:1d:6e:05:b5:c3:47:c7:
    59:34:80:bb:97:fe:ce:74:bd:eb:0f:38:37:3b:86:
    14:da:95:5a:0c:10:60:bb:16:50:28:da:5a:bb:7b:
    01:14:2f:6d:54:66:29:70:58:9e:fe:55:70:4f:27:
    85:6f:55:8f:99:38:3b:a0:d5:92:b7:1c:8a:61:15:
    85:fa:d9:48:fa:1c:fd:5a:43:e3:c3:36:67:24:97:
    09:be:53:ad:46:2d:2a:ec:fc:90:eb:91:98:9c:08:
    8c:92:7c:c9:b3:94:d3:8f:5c:b7:86:53:a3:fa:b7:
    92:99:f9:1a:9e:3e:05:c4:6b:b7:c4:74:ca:1e:e2:
    77:dd:73:b7:33:9f:ec:94:51:c9:09:6f:40:af:f6:
    05:19:01:f2:a6:be:46:98:ff:61:aa:27:0b:21:9c:
    d6:f8:b6:45:c8:e5:a3:d3:f3:97:81:df:0f:ef:d0:
    df:47
Exponent: 3 (0x3)


In [40]:
!openssl dgst -sha1 -verify pubkey_ToJa.pem -signature signature1.bin plik1.txt
!openssl dgst -sha1 -verify pubkey_ToJa.pem -signature signature2.bin plik2.txt
!openssl dgst -sha1 -verify pubkey_ToJa.pem -signature signature3.bin plik3.txt


Verified OK
Verification failure
40371A513E7F0000:error:0200008A:rsa routines:RSA_padding_check_PKCS1_type_1:invalid padding:../crypto/rsa/rsa_pk1.c:79:
40371A513E7F0000:error:02000072:rsa routines:rsa_ossl_public_decrypt:padding check failed:../crypto/rsa/rsa_ossl.c:697:
40371A513E7F0000:error:1C880004:Provider routines:rsa_verify:RSA lib:../providers/implementations/signature/rsa_sig.c:774:
Verification failure
40A775410B7F0000:error:0200008A:rsa routines:RSA_padding_check_PKCS1_type_1:invalid padding:../crypto/rsa/rsa_pk1.c:79:
40A775410B7F0000:error:02000072:rsa routines:rsa_ossl_public_decrypt:padding check failed:../crypto/rsa/rsa_ossl.c:697:
40A775410B7F0000:error:1C880004:Provider routines:rsa_verify:RSA lib:../providers/implementations/signature/rsa_sig.c:774:


## Zadanie 7 (2 pkt.)

W tym zadaniu przećwiczymy bardziej pełny schemat komunikacji. Stwórz trzy foldery **Alice**, **Bob**, **CA**. Alicja będzie nadawcą pliku, Bob odbiorcą a CA to zaufana trzecia strona.

Użyj następujących algorytmów:
- RSA 2048 e=65537
- SHA 256
- AES 256 CBC z kluczem generowanym z hasła

1. Każda ze stron generuje swoje pary klucz prywatny **privkey_Alice.pem** i klucz publiczny **pubkey_Alice.pem**.
2. W ogólości klucz prywatny należy zaszyfrować jakimś szyfrem symetrycznym, np. **AES** aby go bezpiecznie przechowywać, tu dla ułatwienia pomieniemy ten krok.
3. Zaufana trzecia strona **CA** tworzy swój certyfikat, podpisany przez samą siebie **root.crt**, który jest ogłaszany publicznie, tzn. trafia do Alicji i Boba.
4. Następnie Alicja i Bob muszą uzyskać certyfikaty podpisane przez **CA**. W związku z tym każde z nich tworzy rządanie certyfikatu **Alice_req.csr**.
5. Alicja i Bob wysyłają swoje żądania do **CA**, które po zweryfikowaniu Alicji i Boba podpisuje je. Podpisane certyfikaty Alicji i Boba sa ogłaszane publicznie.
6. Alicja bierze certyfikat Boba weryfikuje go za pomocą certyfikatu **CA** (**root.crt**). Jeśli certyfikat jest poprawny, wyciąga z niego klucz publiczny Boba i zapisuje do pliku **pubkey_Bob.pem**.
7. Alicja generuje losowe hasło (**sympass.pem**) z którego będzie generowany klucz symetryczny, które będzie chciała przekazać Bobowi.
8. Alicja szyfruje hasło **sympass.pem** za pomocą klucza publicznego Boba **pubkey_Bob.pem** otrzymując **sympass.enc**.
7. Alicja podpisuje zaszyfrowane hasło poprzez zhashowanie go i zaszyfrowanie za pomocą swojego klucza prywatnego **privkey_ALice.pem**. Podpis zapisuje w pliku **signature.bin**.
8. Alicja wysyła do Boba zaszyfrowane hasło **sympass.enc** oraz jego podpis **signature.bin**.
9. Bob weryfikuje certyfikat Alicji, wyciąga z niego jej klucz publiczny, deszyfruje otrzymane hasło **sympass.enc** i sprawdza podpis.
10. Alicja szyfruje plik, który chce przekazać Bobowi za pomocą hasła **sympass.pem** i wysyła zaszyfrowany plik do Boba.
11. Bob deszyfruje otrzymany plik za pomocą **sympass.pem**.

In [55]:
!rm -rf Alice Bob CA
!mkdir Alice Bob CA
!ls -la


total 124
drwxrwxrwx 1 ziolo ziolo  4096 Jan 13 22:32 .
drwxrwxrwx 1 ziolo ziolo  4096 Jan 13 20:42 ..
-rwxrwxrwx 1 ziolo ziolo  3388 Jan 13 21:47 AESencrypted.dec
-rwxrwxrwx 1 ziolo ziolo  3392 Jan 13 21:32 AESencrypted.enc
drwxrwxrwx 1 ziolo ziolo  4096 Jan 13 22:32 Alice
drwxrwxrwx 1 ziolo ziolo  4096 Jan 13 22:32 Bob
drwxrwxrwx 1 ziolo ziolo  4096 Jan 13 22:32 CA
-rwxrwxrwx 1 ziolo ziolo  1592 Jan 13 22:01 E8.crt
-rwxrwxrwx 1 ziolo ziolo 61694 Jan 13 22:32 Lista11.ipynb
-rwxrwxrwx 1 ziolo ziolo  1046 Jan 13 20:42 ToJa1.crt
-rwxrwxrwx 1 ziolo ziolo  1046 Jan 13 20:42 ToJa2.crt
-rwxrwxrwx 1 ziolo ziolo  1078 Jan 13 20:42 ToJa3.crt
-rwxrwxrwx 1 ziolo ziolo     0 Jan 13 22:21 bajka.rsa.enc
-rwxrwxrwx 1 ziolo ziolo  2352 Jan 13 20:41 bajka.txt
-rwxrwxrwx 1 ziolo ziolo   424 Jan 13 21:50 dh_params.pem
-rwxrwxrwx 1 ziolo ziolo    21 Jan 13 21:47 haslo.txt
-rwxrwxrwx 1 ziolo ziolo   928 Jan 13 21:47 kamien.dec.txt
-rwxrwxrwx 1 ziolo ziolo   944 Jan 13 21:47 kamien.ofb.enc
-rwxrwxrwx 1 ziol

In [56]:
# Alice RSA
%cd Alice
!openssl genpkey -algorithm RSA -pkeyopt rsa_keygen_bits:2048 -out privkey_Alice.pem
!openssl pkey -in privkey_Alice.pem -pubout -out pubkey_Alice.pem
%cd ..

# Bob RSA
%cd Bob
!openssl genpkey -algorithm RSA -pkeyopt rsa_keygen_bits:2048 -out privkey_Bob.pem
!openssl pkey -in privkey_Bob.pem -pubout -out pubkey_Bob.pem
%cd ..

# CA RSA
%cd CA
!openssl genpkey -algorithm RSA -pkeyopt rsa_keygen_bits:2048 -out rootkey.pem
!openssl pkey -in rootkey.pem -pubout -out rootpub.pem
%cd ..


/mnt/d/Studia/Kryptografia/zestaw11/Alice
...+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*.+.+...+..+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*..........................+.....+...+....+..+....+.........+..+..........+...+........+...+.+...+......+......+........+.............+..............+.........+......+.........+......+.+..+.+...........+...+...+..........+..+...............+..................+..........+..+...+.......+........+...+.......+...+......+.........+.........+.....+...+.........+.+.........+..............+......+.+..+.........................+........+....+...+..+...+...+....+...+...+...............+..+......+.+.....+...+.+...........+.........+.......+........+......+....+...............+..+...................+...+...+.................+.+..+....+.....+...+......+....+........+.......+...+..+.....................+..........+..+.......+....................+.......+...........+...+......+....+.........+.....+...+.........+...+.+

In [57]:
%cd CA
!openssl req -x509 -new -nodes -key rootkey.pem -sha256 -days 1024 -out root.crt -subj "/C=PL/ST=MAZ/O=CA-UJ/CN=RootCA"
%cd ..


/mnt/d/Studia/Kryptografia/zestaw11/CA
/mnt/d/Studia/Kryptografia/zestaw11


In [58]:
!cp CA/root.crt Alice/
!cp CA/root.crt Bob/


In [59]:
# Alice CSR
%cd Alice
!openssl req -new -key privkey_Alice.pem -out Alice_req.csr -subj "/C=PL/ST=MAZ/O=Alice/CN=Alice"
%cd ..

# Bob CSR
%cd Bob
!openssl req -new -key privkey_Bob.pem -out Bob_req.csr -subj "/C=PL/ST=MAZ/O=Bob/CN=Bob"
%cd ..


/mnt/d/Studia/Kryptografia/zestaw11/Alice
/mnt/d/Studia/Kryptografia/zestaw11
/mnt/d/Studia/Kryptografia/zestaw11/Bob
/mnt/d/Studia/Kryptografia/zestaw11


In [60]:
!cp Alice/Alice_req.csr CA/
!cp Bob/Bob_req.csr CA/

%cd CA
!openssl x509 -req -in Alice_req.csr -CA root.crt -CAkey rootkey.pem -CAcreateserial -out Alice.crt -days 500 -sha256
!openssl x509 -req -in Bob_req.csr   -CA root.crt -CAkey rootkey.pem -CAcreateserial -out Bob.crt   -days 500 -sha256
!ls -la Alice.crt Bob.crt
%cd ..

# publikacja certyfikatów
!cp CA/Alice.crt Alice/
!cp CA/Bob.crt Bob/
!cp CA/Alice.crt Bob/
!cp CA/Bob.crt Alice/


/mnt/d/Studia/Kryptografia/zestaw11/CA
Certificate request self-signature ok
subject=C = PL, ST = MAZ, O = Alice, CN = Alice
Certificate request self-signature ok
subject=C = PL, ST = MAZ, O = Bob, CN = Bob
-rwxrwxrwx 1 ziolo ziolo 1099 Jan 13 22:33 Alice.crt
-rwxrwxrwx 1 ziolo ziolo 1094 Jan 13  2026 Bob.crt
/mnt/d/Studia/Kryptografia/zestaw11


In [61]:
%cd Alice
!openssl verify -CAfile root.crt Bob.crt
!openssl x509 -pubkey -in Bob.crt -noout > pubkey_Bob.pem
%cd ..


/mnt/d/Studia/Kryptografia/zestaw11/Alice
Bob.crt: OK
/mnt/d/Studia/Kryptografia/zestaw11


In [67]:
%cd Alice

!openssl rand -out sympass.pem -base64 32

!ls -la sympass.pem
!head -c 80 sympass.pem; echo

# RSA-encrypt sympass dla Boba
!openssl pkeyutl -encrypt -pubin -inkey pubkey_Bob.pem -in sympass.pem -out sympass.enc

# Podpis zaszyfrowanego hasła (SHA256 + RSA)
!openssl dgst -sha256 -sign privkey_Alice.pem -out signature.bin sympass.enc

!ls -la sympass.pem sympass.enc signature.bin
%cd ..


/mnt/d/Studia/Kryptografia/zestaw11/Alice
-rwxrwxrwx 1 ziolo ziolo 45 Jan 13  2026 sympass.pem
ffZjmGSnuFThW8h71sjAFHiy/l26PiWwVjpsCZZfGi4=

-rwxrwxrwx 1 ziolo ziolo 256 Jan 13  2026 signature.bin
-rwxrwxrwx 1 ziolo ziolo 256 Jan 13 22:36 sympass.enc
-rwxrwxrwx 1 ziolo ziolo  45 Jan 13 22:36 sympass.pem
/mnt/d/Studia/Kryptografia/zestaw11


In [68]:
!cp Alice/sympass.enc Bob/
!cp Alice/signature.bin Bob/


In [69]:
%cd Bob
!openssl verify -CAfile root.crt Alice.crt
!openssl x509 -pubkey -in Alice.crt -noout > pubkey_Alice.pem

# weryfikacja podpisu
!openssl dgst -sha256 -verify pubkey_Alice.pem -signature signature.bin sympass.enc

# odszyfrowanie hasła
!openssl pkeyutl -decrypt -inkey privkey_Bob.pem -in sympass.enc -out sympass.pem

!ls -la sympass.pem
%cd ..


/mnt/d/Studia/Kryptografia/zestaw11/Bob
Alice.crt: OK
Verified OK
-rwxrwxrwx 1 ziolo ziolo 45 Jan 13 22:37 sympass.pem
/mnt/d/Studia/Kryptografia/zestaw11


In [70]:
!cp rozwijajac_rilkego.txt Alice/

%cd Alice
!openssl enc -aes-256-cbc -pbkdf2 -salt -in rozwijajac_rilkego.txt -out msg.enc -pass file:sympass.pem
%cd ..

!cp Alice/msg.enc Bob/


/mnt/d/Studia/Kryptografia/zestaw11/Alice
/mnt/d/Studia/Kryptografia/zestaw11


In [71]:
%cd Bob
!openssl enc -aes-256-cbc -d -pbkdf2 -in msg.enc -out msg.dec.txt -pass file:sympass.pem

!diff -u ../rozwijajac_rilkego.txt msg.dec.txt || true
%cd ..


/mnt/d/Studia/Kryptografia/zestaw11/Bob
/mnt/d/Studia/Kryptografia/zestaw11
